<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angelo Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Benjamín Torrealba 
- Nombre de alumno 2: Israel Rodríguez


### **Link de repositorio de GitHub:** https://github.com/BnjmnNicholas/MDS7202-2023-2

## Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

### **Link de repositorio de GitHub:** `http://....`

# Importamos librerias útiles

In [250]:
#!pip install -qq xgboost optuna

In [251]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
import joblib

In [252]:
import optuna
from optuna.samplers import TPESampler
import optuna
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

# 1. El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [253]:
df = pd.read_csv('sales.csv')
df['date'] = pd.to_datetime(df['date'])

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1.1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

#### 1. Split Data

In [254]:
#   Creamos los conjuntos Train 70%, Validation 20% y Test 10%
train, test = train_test_split(df, test_size=0.1, random_state=99)
train, val = train_test_split(train, test_size=0.2, random_state=42) 
X_train = train.drop(['quantity','id'], axis=1)
y_train = train['quantity']
X_val = val.drop(['quantity','id'], axis=1)
y_val = val['quantity']
X_test = test.drop(['quantity','id'], axis=1)
y_test = test['quantity']


#### 2. Date Transformation

In [255]:
def date_to_categorical(data):
    """
    Toma como entrada un dataset de una columna y retorna dia, mes y año como columnas separadas
    
    Args:
        df (DataFrame): Dataset con una columna de tipo fecha
    
    Returns:
        DataFrame: Dataset con tres columnas nuevas: dia, mes y año
    """
    data['date'] = pd.to_datetime(data['date'])
    data['day'] = data['date'].dt.day.astype('category')
    data['month'] = data['date'].dt.month.astype('category')
    data['year'] = data['date'].dt.year.astype('category')
    return data.drop('date', axis=1)

date_transformer = FunctionTransformer(date_to_categorical)

#### 3. ColumnTransformer

In [256]:
def numerical(data):
    """
    Toma como entrada un dataset y retorna las columnas numéricas en formato lista
    
    Args:
        df (DataFrame): Dataset
    
    Returns:
        DataFrame: Dataset con las columnas numéricas
    """
    return data.select_dtypes(include=['int64', 'float64']).columns.tolist()

def categorical(data):
    """
    Toma como entrada un dataset y retorna las columnas categóricas en formato lista
    
    Args:
        df (DataFrame): Dataset
    
    Returns:
        DataFrame: Dataset con las columnas categóricas
    """
    return data.select_dtypes(include=['object', 'category']).columns.tolist()

In [257]:
num_cols = ['lat', 'long', 'pop', 'price']
cat_cols = ['city', 'shop', 'brand', 'container', 'capacity', 'day', 'month', 'year']

In [258]:
# Creamos el ColumnTransformer que procesa los datos numericos y categoricos
preprossesor = ColumnTransformer([
    ('numerical', MinMaxScaler(), num_cols),
    ('categoric', OneHotEncoder(), cat_cols)
])

In [259]:
pipeline_val = Pipeline([
        ('date_transformer', date_transformer),
        ('preprossesor', preprossesor),
        ('create_column_names', FunctionTransformer(
        lambda X: pd.DataFrame(X.toarray(), columns=num_cols + preprossesor.transformers_[1][1].get_feature_names_out().tolist())
    ))
])
pipeline_val

Pipeline(steps=[('date_transformer',
                 FunctionTransformer(func=<function date_to_categorical at 0x000001D42FC80720>)),
                ('preprossesor',
                 ColumnTransformer(transformers=[('numerical', MinMaxScaler(),
                                                  ['lat', 'long', 'pop',
                                                   'price']),
                                                 ('categoric', OneHotEncoder(),
                                                  ['city', 'shop', 'brand',
                                                   'container', 'capacity',
                                                   'day', 'month', 'year'])])),
                ('create_column_names',
                 FunctionTransformer(func=<function <lambda> at 0x000001D42FC83BA0>))])

In [260]:
X_val

,date,city,lat,long,pop,shop,brand,container,capacity,price
2037,2013-11-30,Athens,37.96245,23.68708,671022,shop_3,gazoza,glass,500ml,0.51
4359,2016-02-29,Irakleion,35.32787,25.14341,137302,shop_2,kinder-cola,can,330ml,0.50
3690,2015-06-30,Athens,37.96245,23.68708,667237,shop_3,lemon-boost,glass,500ml,1.09
2752,2014-08-31,Athens,37.97945,23.71622,668203,shop_1,orange-power,can,330ml,0.42
1412,2013-04-30,Patra,38.24444,21.73444,166301,shop_6,kinder-cola,plastic,1.5lt,3.42
...,...,...,...,...,...,...,...,...,...,...
6470,2018-02-28,Athens,37.97945,23.71622,664046,shop_1,adult-cola,plastic,1.5lt,2.58
293,2012-04-30,Patra,38.24444,21.73444,164250,shop_6,adult-cola,can,330ml,0.65
4577,2016-04-30,Athens,37.96245,23.68708,665102,shop_3,orange-power,glass,500ml,0.86
2830,2014-08-31,Patra,38.24444,21.73444,167242,shop_6,gazoza,glass,500ml,0.69


In [261]:
X_val_2 = pipeline_val.fit_transform(X_val)
X_val_2

,lat,long,pop,price,city_Athens,city_Irakleion,city_Larisa,city_Patra,city_Thessaloniki,shop_shop_1,...,month_10,month_11,month_12,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,0.495619,0.572795,0.997940,0.079823,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,1.000000,0.005731,0.077605,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.495619,0.572795,0.990904,0.208426,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.498817,0.581343,0.992700,0.059867,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.548667,0.000000,0.059642,0.725055,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337,0.498817,0.581343,0.984971,0.538803,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1338,0.548667,0.000000,0.055829,0.110865,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1339,0.495619,0.572795,0.986935,0.157428,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1340,0.548667,0.000000,0.061391,0.119734,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


#### 4. Dummy 

In [262]:
pipeline_dummy = Pipeline([
        ('date_transformer', date_transformer),
        ('preprossesor', preprossesor),
        ('create_column_names', FunctionTransformer(
        lambda X: pd.DataFrame(X.toarray(), columns=num_cols + preprossesor.transformers_[1][1].get_feature_names_out().tolist())
    )),
        ('regressor', DummyRegressor())
])
pipeline_dummy

Pipeline(steps=[('date_transformer',
                 FunctionTransformer(func=<function date_to_categorical at 0x000001D42FC80720>)),
                ('preprossesor',
                 ColumnTransformer(transformers=[('numerical', MinMaxScaler(),
                                                  ['lat', 'long', 'pop',
                                                   'price']),
                                                 ('categoric', OneHotEncoder(),
                                                  ['city', 'shop', 'brand',
                                                   'container', 'capacity',
                                                   'day', 'month', 'year'])])),
                ('create_column_names',
                 FunctionTransformer(func=<function <lambda> at 0x000001D42FCECFE0>)),
                ('regressor', DummyRegressor())])

#### 5. Train Dummy

In [263]:
# 5. Entrena el Pipeline y evalúa en datos de validación


pipeline_dummy.fit(X = X_train, y=y_train)
val_predictions_dummy = pipeline_dummy.predict(X_val)
mae_dummy = mean_absolute_error(y_val, val_predictions_dummy)
print(f'MAE Dummy: {mae_dummy}')


MAE Dummy: 13600.590469326427


#### 6. XGB

In [264]:
pipeline_xgb = Pipeline([
        ('data_transformer', date_transformer),
        ('preprossesor', preprossesor),
        ('create_column_names', FunctionTransformer(
        lambda X: pd.DataFrame(X.toarray(), columns=num_cols + preprossesor.transformers_[1][1].get_feature_names_out().tolist())
    )),
        ('regressor', XGBRegressor())
])
pipeline_xgb

Pipeline(steps=[('data_transformer',
                 FunctionTransformer(func=<function date_to_categorical at 0x000001D42FC80720>)),
                ('preprossesor',
                 ColumnTransformer(transformers=[('numerical', MinMaxScaler(),
                                                  ['lat', 'long', 'pop',
                                                   'price']),
                                                 ('categoric', OneHotEncoder(),
                                                  ['city', 'shop', 'brand',
                                                   'container', 'capacity',
                                                   'day', 'month', 'year'])])),
                ('create_column_names',
                 Functio...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [265]:
# 6. XGB

pipeline_xgb.fit(X = X_train, y=y_train)
val_predictions_xgb = pipeline_xgb.predict(X_val)
mae_xgb = mean_absolute_error(y_val, val_predictions_xgb)
print(f'MAE XGB: {mae_xgb}')

MAE XGB: 2517.7880235379216


#### 7. Save models

In [266]:
# 7. Guarda los modelos en archivos .pkl
#joblib.dump(pipeline_dummy, 'dummy_model.pkl')
#joblib.dump(pipeline_xgb, 'xgb_model.pkl')

#### 8. Respuestas

- 5. Esta métrica se define como la varianza media entre el valor predicho y el valor real. En el contexto del negocio, representa que tanto se está desviando el modelo en las predicciones, es decir la demanda esperada vs la demanda real. 

- 6. La disminución del error en aproximadamente 11,100 puntos indica una mejora en el rendimiento del `GBXregressor`. Este fenómeno podría atribuirse al hecho de que el `DummyRegressor` utiliza reglas simples para sus predicciones, prediciendo por defecto la mediana de los valores en todos los casos. En consecuencia, cualquier otro regresor debería exhibir un rendimiento superior.

## 1.2 Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo.

Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?

Nuevamente, guarde su modelo en un archivo .pkl

Nota: Para realizar esta parte, debe apoyarse en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>.

Hint: Para implementar el constraint, se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

In [267]:
pipeline_constraints = Pipeline([
        ('date_transformer', date_transformer),
        ('preprossesor', preprossesor),
        ('create_column_names', FunctionTransformer(
        lambda X: pd.DataFrame(X.toarray(), columns=num_cols + preprossesor.transformers_[1][1].get_feature_names_out().tolist())
    )),
        ('regressor', XGBRegressor(interaction_constraints=[{"price":-1}]))
])


pipeline_constraints.fit(X = X_train, y=y_train)
val_predictions_xgb = pipeline_constraints.predict(X_val)
mae_xgb = mean_absolute_error(y_val, val_predictions_xgb)
print(f'MAE XGB with Constraints: {mae_xgb}')

MAE XGB with Constraints: 5173.394713684628


#### Respuesta:

En relación con los resultados de la sección anterior, donde se aplicó el regresor sin imponer condiciones sobre las variables, se observa un desempeño inferior, casi el doble en términos de error. Por lo tanto, la afirmación del amigo no resultó válida en este contexto.

Esto podría explicarse por la existencia de otras relaciones en el modelo que se ajustan de manera más efectiva a los datos, lo cual destaca la importancia de considerar diversas condiciones en el aprendizaje del modelo.




## 1.3 Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

#### Explicación hiperparametros

`learning_rate` : Este hiperparametro controla la tasa de aprendizaje del modelo y en el contexto de XGBRegressor tambien controla la contribución de cada árbol al modelo final. Valores pequeños hacen que el entrenamiento sea más lento requiriendo más iteraciones pero pudiendo obtener mejores resultados con datos no vistos a diferencia de valores grandes que tiene un impacto opuesto.

`n_estimators` : Indica el número de árboles que se construyen. Si se utiliza un valor alto generalmente se obtiene un mejor rendimiento. Es importante ajustar este hiperparametro en conjunto con el `learning_rate` para lograr un equilibrio entre rendimiento y eficiencia computacional.

`max_depth` : Indica la profundidad máxima de cada árbol controlando la complejidad del modelo. Valores altos permite capturar relaciones complejas en los datos pero aumentando el riesgo de sobreajuste.

`max_leaves` : Indica el número máximo de hojas en cada árbol. Cumple la misma función que `max_depth`.

`min_child_weight` : Este parámetro controla la cantidad mínima de instancias de muestra (o suma mínima de pesos) necesarias en un nodo hoja. Valores bajos permiten particiones más finas pudiendo capturar relaciones más detalladas pero puede hacer que el modelo sea sensible a datos atípicos o ruidosos, en cambio, para valores altos se promueve la creación de nodos hojas más generalizados y robustos pero podria resultar en un modelo que no capture relaciones más detalladas en los datos.

`reg_alpha` : Regula la magnitud absoluta de los pesos de las hojas. Valores bajos permite que los pesos de las hojas sean grandes, es decir, permite asignar más importancia a características especificas pero tiene la desventaja de hacer que el modelo sea más sensible a los datos ruidosos del entrenamiento, en cambio los valores altos penalizan fuertemente los pesos de las hojas, favoreciendo la simplicidad al evitar que el modelo dependa de características específicas teniendo la desventaja de que el modelo puede no capturar correctamente algunas relaciones entre los datos.

`reg_lambda` : Tiene el mismo rol que `reg_alpha` con la diferencia que para la penalización utiliza la norma euclidiana. El uso de valores altos o bajos tiene un impacto similar al de `reg_alpha`.

Respecto a los intervalos, todos parecen valores razonables y de uso común a excepción de `max_leaves` donde este parte de 0, utilizar `max_leaves = 0` significa que no se permite la creación de hojas, lo cual no tiene sentido en el contexto del problema.

Pasando al hiperparámetro `min_frecuency` de OneHotEncoder, este especifica la frecuencia minima para que una categoria sea considerada infrecuente, es decir, si tiene una frecuencia menor a `min_frecuency` entonces es considerada infrecuente. En el caso de utilizar un valor entre 0 y 1, la frecuencia minima sera `num_samples` * `min_frecuency`. Finalmente, el intervalo escogido para la optimización tiene sentido con el unico detalle de que no se debe considerar valores muy cercanos a 0 ni muy cercanos a 1.

In [284]:
# Inserte su código acá
def objective_function(trial):
    #X = df.drop(columns=["id"]).astype(float)

    # Define the hyperparameters to tune
    params = {
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.001, 0.1),
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "max_leaves": trial.suggest_int("max_leaves", 0, 100),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 5),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 1.0),
    }

    params_encoder = {
        "min_frequency": trial.suggest_float("min_frequency", 0.1, 0.99),
    }

    preprossesor_optuna = ColumnTransformer([
    ('numerical', MinMaxScaler(), num_cols),
    ('categoric', OneHotEncoder(**params_encoder), cat_cols)
    ])

    pipeline_optuna = Pipeline([
        ('date_transformer', date_transformer),
        ('preprossesor', preprossesor_optuna),
        ('create_column_names', FunctionTransformer(
        lambda X: pd.DataFrame(X, columns=num_cols + preprossesor_optuna.transformers_[1][1].get_feature_names_out().tolist())
    )),
        ("model",XGBRegressor(seed=42, **params))
    ])

    pipeline_optuna.fit(X = X_train, y=y_train)

    # Predict and evaluate the model
    yhat = pipeline_optuna.predict(X_val)
    mae = mean_absolute_error(y_val, yhat)

    return mae

In [285]:
study = optuna.create_study(direction="minimize", sampler=TPESampler(seed=42))
study.optimize(objective_function, timeout=300, n_trials=500)

C:\Users\Benjamin Nicholas\AppData\Local\Temp\ipykernel_26364\3963847990.py:7: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

C:\Users\Benjamin Nicholas\AppData\Local\Temp\ipykernel_26364\3963847990.py:7: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

C:\Users\Benjamin Nicholas\AppData\Local\Temp\ipykernel_26364\3963847990.py:7: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

C:\Users\Benjamin Nicholas\AppData\Local\Temp\ipykernel_26364\3963847990.py:7: FutureWarning:

suggest_loguniform has been deprecated in v3.0.

In [286]:
study.best_params

{'learning_rate': 0.042334583724474656,
 'n_estimators': 419,
 'max_depth': 9,
 'max_leaves': 18,
 'min_child_weight': 4,
 'reg_alpha': 0.6967270897897572,
 'reg_lambda': 0.22451747833860441,
 'min_frequency': 0.14926348957619065}

In [287]:
plot_optimization_history(study)

In [288]:
plot_parallel_coordinate(study)

In [289]:
plot_param_importances(study)

In [291]:
trials_without_prunning = study.trials_dataframe()
trials_without_prunning

,number,value,datetime_start,datetime_complete,duration,params_learning_rate,params_max_depth,params_max_leaves,params_min_child_weight,params_min_frequency,params_n_estimators,params_reg_alpha,params_reg_lambda,state
0,0,8486.993779,2023-11-17 22:28:47.463756,2023-11-17 22:28:49.544283,0 days 00:00:02.080527,0.005612,8,60,1,0.870897,954,0.155995,0.058084,COMPLETE
1,1,6474.199834,2023-11-17 22:28:49.544283,2023-11-17 22:28:50.315055,0 days 00:00:00.770772,0.015931,3,97,5,0.263230,723,0.212339,0.181825,COMPLETE
2,2,8583.263719,2023-11-17 22:28:50.316054,2023-11-17 22:28:51.362350,0 days 00:00:01.046296,0.004060,6,29,4,0.426062,549,0.139494,0.292145,COMPLETE
3,3,6482.960070,2023-11-17 22:28:51.362350,2023-11-17 22:28:52.607464,0 days 00:00:01.245114,0.008168,4,51,3,0.251766,796,0.046450,0.607545,COMPLETE
4,4,9077.405621,2023-11-17 22:28:52.608461,2023-11-17 22:28:54.792399,0 days 00:00:02.183938,0.001349,10,81,2,0.491736,952,0.097672,0.684233,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,399,6433.285110,2023-11-17 22:33:43.812417,2023-11-17 22:33:44.648180,0 days 00:00:00.835763,0.043717,9,21,4,0.221879,472,0.746895,0.226631,COMPLETE
400,400,5665.602391,2023-11-17 22:33:44.650178,2023-11-17 22:33:45.551730,0 days 00:00:00.901552,0.056403,9,16,4,0.101255,355,0.717644,0.200252,COMPLETE
401,401,5666.436091,2023-11-17 22:33:45.552235,2023-11-17 22:33:46.319438,0 days 00:00:00.767203,0.039034,8,18,4,0.125986,377,0.674451,0.322795,COMPLETE
402,402,5639.280544,2023-11-17 22:33:46.319438,2023-11-17 22:33:47.166308,0 days 00:00:00.846870,0.030845,9,22,4,0.183704,535,0.606941,0.354681,COMPLETE


#### Respuestas:

- El número de trials fue 404, esto pues se cumplió el tiempo límite de ejecución. Los mejores hiperparámetros encontrados son:

    - 'learning_rate': 0.042
    - 'n_estimators': 419
    - 'max_depth': 9
    - 'max_leaves': 18
    - 'min_child_weight': 4
    - 'reg_alpha': 0.69
    - 'reg_lambda': 0.22
    - 'min_frequency': 0.14

Con respecto a la optimización anterior, el error MAE no mejora, se tiene un valor cercano a 5600. 

Esto se puede deber a la cantidad limitada de intentos, el espacio de optimización, el rango acotado de hiperparámetros.  

- Posibles alternativas de mejora podrían ser:

    - Aumentar la cantidad de intentos
    - Reducir o aumentar el espacio de optimización (cantidad de hiperparámetros)
    - Modificar los rangos de los hiperparámetros.
    - Probar otras semillas.
    

## 1.4 Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

#### Respuestas:

- Prunning: El 'prunning' (poda en español) en su concepto más general hace relación a la eliminación de procesos u objetos innecesarios. En este caso es una característica de la optimización de Optuna, la cual permite detener de manera anticipada aquellos intentos que no proveeran buenos resultados. Esto tiene efectos positivos en el entrenamiento ya que ahorra recursos y tiempo de optimización. 

In [275]:
# Inserta tu código aquí
def objective_function_pruning(trial):
    # Define the hyperparameters to tune
    params = {
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.001, 0.1),
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "max_leaves": trial.suggest_int("max_leaves", 0, 100),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 5),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 1.0),
        "eval_metric": "rmse",
    }

    params_encoder = {
        "min_frequency": trial.suggest_float("min_frequency", 0.1, 0.99),
    }

    # Pruning
    pruning_callback = optuna.integration.XGBoostPruningCallback(
        trial, "validation_0-rmse"
    )

    # Definir el preprocesamiento como un pipeline separado
    preprocessor = ColumnTransformer([
        ('numerical', MinMaxScaler(), num_cols),
        ('categoric', OneHotEncoder(**params_encoder), cat_cols)
    ])

    # Entrenar el preprocesador
    X_train_transformed = preprocessor.fit_transform(X_train)

    # Crear el conjunto de entrenamiento con las características transformadas
    X_train_processed = pd.DataFrame(X_train_transformed, columns=num_cols + preprocessor.transformers_[1][1].get_feature_names_out().tolist())
    
    X_val_transformed = preprocessor.transform(X_val)
    
    X_val_processed = pd.DataFrame(X_val_transformed, columns=num_cols + preprocessor.transformers_[1][1].get_feature_names_out().tolist())

    # Definir el modelo como un pipeline separado
    model = XGBRegressor(seed=42, **params)

    # Entrenar el modelo
    model.fit(X_train_processed, y_train, callbacks=[pruning_callback], eval_set=[(X_train_processed, y_train), (X_val_processed, y_val)])

    # Predict y evaluar el modelo
    X_val_transformed = preprocessor.transform(X_val)
    yhat = model.predict(X_val_transformed)
    mae = mean_absolute_error(y_val, yhat)

    return mae



In [276]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [ ]:
study_prunning = optuna.create_study(direction="minimize", sampler=TPESampler(seed=42))
study_prunning.optimize(objective_function_pruning, timeout=300, n_trials=500, show_progress_bar=True)

In [278]:
study_prunning.best_params

{'learning_rate': 0.09990142136171117,
 'n_estimators': 105,
 'max_depth': 9,
 'max_leaves': 39,
 'min_child_weight': 2,
 'reg_alpha': 0.8555224130755674,
 'reg_lambda': 0.10419955152993479,
 'min_frequency': 0.15242023996226886}

In [279]:
plot_optimization_history(study_prunning)

In [280]:
plot_parallel_coordinate(study_prunning)

In [281]:
plot_param_importances(study_prunning)

In [296]:
trials_with_prunning = study_prunning.trials_dataframe()
trials_with_prunning

,number,value,datetime_start,datetime_complete,duration,params_learning_rate,params_max_depth,params_max_leaves,params_min_child_weight,params_min_frequency,params_n_estimators,params_reg_alpha,params_reg_lambda,state
0,0,8486.993779,2023-11-17 22:08:17.556790,2023-11-17 22:08:22.957064,0 days 00:00:05.400274,0.005612,8,60,1,0.870897,954,0.155995,0.058084,COMPLETE
1,1,6474.199834,2023-11-17 22:08:22.968140,2023-11-17 22:08:25.956392,0 days 00:00:02.988252,0.015931,3,97,5,0.263230,723,0.212339,0.181825,COMPLETE
2,2,8583.263719,2023-11-17 22:08:25.964440,2023-11-17 22:08:28.237388,0 days 00:00:02.272948,0.004060,6,29,4,0.426062,549,0.139494,0.292145,COMPLETE
3,3,6482.960070,2023-11-17 22:08:28.246442,2023-11-17 22:08:31.561388,0 days 00:00:03.314946,0.008168,4,51,3,0.251766,796,0.046450,0.607545,COMPLETE
4,4,9077.405621,2023-11-17 22:08:31.569417,2023-11-17 22:08:36.309059,0 days 00:00:04.739642,0.001349,10,81,2,0.491736,952,0.097672,0.684233,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,17176.613277,2023-11-17 22:12:58.645295,2023-11-17 22:12:58.863748,0 days 00:00:00.218453,0.034125,9,51,2,0.147333,117,0.062149,0.643544,PRUNED
496,496,17373.350582,2023-11-17 22:12:58.867337,2023-11-17 22:12:59.090254,0 days 00:00:00.222917,0.020232,8,37,2,0.707956,236,0.629177,0.686615,PRUNED
497,497,16660.211842,2023-11-17 22:12:59.096300,2023-11-17 22:12:59.385303,0 days 00:00:00.289003,0.092878,10,40,2,0.655295,83,0.780058,0.181670,PRUNED
498,498,17260.557586,2023-11-17 22:12:59.389302,2023-11-17 22:12:59.652407,0 days 00:00:00.263105,0.038003,3,44,2,0.181678,50,0.800435,0.124983,PRUNED


In [300]:
print('Número de intentos completados: ', trials_with_prunning.loc[trials_with_prunning['state'] == 'COMPLETE'].sort_values(by='value').count().value)

Número de intentos completados:  117


#### Respuestas:

- El número de trials fue 500, de los cuales 383 fueron cancelados en el `Prunning_callback`. Los mejores hiperparámetros encontrados son:
    - 'learning_rate': 0.099
    - 'n_estimators': 105
    - 'max_depth': 9
    - 'max_leaves': 39
    - 'min_child_weight': 2
    - 'reg_alpha': 0.85
    - 'reg_lambda': 0.10
    - 'min_frequency': 0.152

Con respecto a la optimización anterior, los valores de hiperparámetros son distintos sin embargo el error MAE converge en ambos casos a un valor cercano a 5600. 

La principal diferencia de las optimizaciones radica en el tiempo y optimización de recursos. En este caso se observó que de la totalidad de intentos, practicamente 1/5 de ellos fueron realizados, esto permitiría aumentar la cantidad de intentos con tal de obtener una optimización mejor. 

Sin embargo, se debe tener en cuenta que en una optimización podría darse el caso en el cual los valores estén oscilando en un rango acotado (mínimo local), lo que generaría que por más que se aumente la cantidad de intentos, no se alcanzará el mínimo global. 

## 1.5 Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

- Gráfico de historial de optimización
- Gráfico de coordenadas paralelas
- Gráfico de importancia de hiperparámetros

Comente sus resultados: ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

## 1.6 Síntesis de resultados (0.3)

Finalmente, genere una tabla resumen del MAE obtenido en los 5 modelos entrenados (desde Baseline hasta XGBoost con Constraints, Optuna y Prunning) y compare sus resultados. ¿Qué modelo obtiene el mejor rendimiento? 

Por último, cargue el mejor modelo, prediga sobre el conjunto de test y reporte su MAE. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>